<a href="https://colab.research.google.com/github/alvumu/TFM/blob/main/SerializacionPacientes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Leemos los archivos JSON

In [ ]:
import json

# Cargar los datos de pacientes y admisiones médicas desde los archivos JSON
with open('patients.json') as f:
    patients_data = json.load(f)

with open('admissions.json') as f:
    admissions_data = json.load(f)

# Crear un diccionario para almacenar la información combinada
combined_data = {}

# Construir un diccionario de pacientes para una búsqueda eficiente
patients_dict = {patient['subject_id']: patient for patient in patients_data}

# Iterar sobre las admisiones médicas y combinar la información del paciente
for admission in admissions_data:
    subject_id = admission['subject_id']
    # Comprobar si el paciente está presente en el JSON de pacientes
    if subject_id in patients_dict:
        # Si el paciente ya está en el diccionario combinado, agregar la admisión médica a su lista
        if subject_id in combined_data:
            combined_data[subject_id]['admissions'].append(admission)
        # Si el paciente no está en el diccionario combinado, crear una nueva entrada
        else:
            combined_data[subject_id] = patients_dict[subject_id]
            combined_data[subject_id]['admissions'] = [admission]

# Serializar la información combinada a texto
combined_json_text = json.dumps(list(combined_data.values()), indent=2)

# Imprimir el resultado
print(combined_json_text)


[
  {
    "subject_id": 10004235,
    "gender": "M",
    "anchor_age": 47,
    "anchor_year": 2196,
    "anchor_year_group": "2014 - 2016",
    "dod": "",
    "admissions": [
      {
        "subject_id": 10004235,
        "hadm_id": 24181354,
        "admittime": "2196-02-24 14:38:00",
        "dischtime": "2196-03-04 14:02:00",
        "deathtime": "",
        "admission_type": "URGENT",
        "admit_provider_id": "P03YMR",
        "admission_location": "TRANSFER FROM HOSPITAL",
        "discharge_location": "SKILLED NURSING FACILITY",
        "insurance": "Medicaid",
        "language": "ENGLISH",
        "marital_status": "SINGLE",
        "race": "BLACK/CAPE VERDEAN",
        "edregtime": "2196-02-24 12:15:00",
        "edouttime": "2196-02-24 17:07:00",
        "hospital_expire_flag": 0
      },
      {
        "subject_id": 10004235,
        "hadm_id": 22187210,
        "admittime": "2196-06-20 21:11:00",
        "dischtime": "2196-06-22 13:30:00",
        "deathtime": "",
   

In [6]:
import json

# Cargar los datos de pacientes y admisiones médicas desde los archivos JSON
with open('patients.json') as f:
    patients_data = json.load(f)

with open('admissions.json') as f:
    admissions_data = json.load(f)

# Crear un diccionario para almacenar la información combinada
combined_data = {}

# Construir un diccionario de pacientes para una búsqueda eficiente
patients_dict = {patient['subject_id']: patient for patient in patients_data}

# Iterar sobre las admisiones médicas y combinar la información del paciente
for admission in admissions_data:
    subject_id = admission['subject_id']
    # Comprobar si el paciente está presente en el JSON de pacientes
    if subject_id in patients_dict:
        # Si el paciente ya está en el diccionario combinado, agregar la admisión médica a su lista
        if subject_id in combined_data:
            combined_data[subject_id]['admissions'].append(admission)
        # Si el paciente no está en el diccionario combinado, crear una nueva entrada
        else:
            combined_data[subject_id] = patients_dict[subject_id]
            combined_data[subject_id]['admissions'] = [admission]

# Construir una cadena de texto para la salida
output_text = ""
for patient_data in combined_data.values():
    output_text += "Patient Info:\n"
    for key, value in patient_data.items():
        if key != 'admissions':
            output_text += f"The {key} of the patient is {value}. "
    for admission in patient_data['admissions']:
        for key, value in admission.items():
            output_text += f"The {key} is {value}. "
        output_text += "\n"
    output_text += "\n"

# Imprimir el resultado
print(output_text)


Patient Info:
The subject_id of the patient is 10004235. The gender of the patient is M. The anchor_age of the patient is 47. The anchor_year of the patient is 2196. The anchor_year_group of the patient is 2014 - 2016. The dod of the patient is . The subject_id is 10004235. The hadm_id is 24181354. The admittime is 2196-02-24 14:38:00. The dischtime is 2196-03-04 14:02:00. The deathtime is . The admission_type is URGENT. The admit_provider_id is P03YMR. The admission_location is TRANSFER FROM HOSPITAL. The discharge_location is SKILLED NURSING FACILITY. The insurance is Medicaid. The language is ENGLISH. The marital_status is SINGLE. The race is BLACK/CAPE VERDEAN. The edregtime is 2196-02-24 12:15:00. The edouttime is 2196-02-24 17:07:00. The hospital_expire_flag is 0. 
The subject_id is 10004235. The hadm_id is 22187210. The admittime is 2196-06-20 21:11:00. The dischtime is 2196-06-22 13:30:00. The deathtime is . The admission_type is DIRECT EMER.. The admit_provider_id is P98TAU. T